In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%%sh
# Download the data - you need to do this only once
wget --no-verbose --output-document=image_cc_1.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_cc_1.jpg
wget --no-verbose --output-document=image_cc_2.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_cc_2.jpg
wget --no-verbose --output-document=image_cc_3.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_cc_3.jpg
wget --no-verbose --output-document=image_bridge.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_bridge.jpg


In [ ]:
class Visualizer():
    def __init__(self, num_rows=1, num_cols=1, figsize=(5,5), axis_off=True, title='', tight=False, cm=None):
        fig, self.axs = plt.subplots(num_rows, num_cols, figsize=figsize, squeeze=False)
        # remove ticks
        if axis_off:
          plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
        # set colormap
        if cm is not None:
            plt.set_cmap(cm)
        # set supertitle
        fig.suptitle(title)
        if tight:
            fig.subplots_adjust(top=0.88)

    def add_image_subplot(self, i, j, image, normalize=False, title_str=''):
        if normalize:
            image = self.normalize_image(image)
        if len(image.shape) == 3:
            #BGR -> RGB
            image = image[:, :, ::-1]
        self.axs[i, j].imshow(image)
        self.axs[i, j].set_title(title_str)

    def add_stem_subplot(self, i, j, x, y, title_str=''):
        self.axs[i, j].stem(x, y)
        self.axs[i, j].set_title(title_str)

    def add_subplot(self, i, j, data, title_str=''):
        self.axs[i, j].plot(data)
        self.axs[i, j].set_title(title_str)

    def add_bar_subplot(self, i, j, x, y, title_str=''):
        self.axs[i, j].bar(x, y)
        self.axs[i, j].set_title(title_str)

    @staticmethod
    def normalize_image(image):
        img = np.float64(image) - np.min(image)
        img /= np.max(img)
        return img

# OpenCV SIFT correspondences example

In [ ]:
# read images
img1 = cv2.imread('image_cc_1.jpg')
img2 = cv2.imread('image_cc_2.jpg')
img3 = cv2.imread('image_cc_3.jpg')
img_bridge = cv2.imread('image_bridge.jpg')

# downsample
img1 = cv2.resize(img1, (0, 0), fx=0.125, fy=0.125)
img2 = cv2.resize(img2, (0, 0), fx=0.125, fy=0.125)
img3 = cv2.resize(img3, (0, 0), fx=0.125, fy=0.125)
img_bridge = cv2.resize(img_bridge, (0, 0), fx=0.125, fy=0.125)

# convert to grayscale for SIFT
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3_gray = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
img_bridge_gray = cv2.cvtColor(img_bridge, cv2.COLOR_BGR2GRAY)

# create sift object
sift = cv2.SIFT_create()

# detect sift features and compute descriptors
kp1, des1 = sift.detectAndCompute(img1_gray, None)
kp2, des2 = sift.detectAndCompute(img2_gray, None)
kp3, des3 = sift.detectAndCompute(img3_gray, None)
kpb, desb = sift.detectAndCompute(img_bridge_gray, None)

# show keypoints
img1_kp = cv2.drawKeypoints(img1_gray, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img2_kp = cv2.drawKeypoints(img2_gray, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img3_kp = cv2.drawKeypoints(img3_gray, kp3, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

vis = Visualizer(1, 3, figsize=(25,5), title='SIFT Keypoints')
vis.add_image_subplot(0, 0, img1_kp)
vis.add_image_subplot(0, 1, img2_kp)
vis.add_image_subplot(0, 2, img3_kp)


In [ ]:
# Matching

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# match descriptors
matches = bf.match(des1, des2)

# sort matches by distance
matches = sorted(matches, key=lambda x: x.distance)

# draw matches
img_matches = cv2.drawMatches(img1_gray, kp1, img2_gray, kp2, matches[:100], None, flags=2)
Visualizer(figsize=(20,5), title='Matches').add_image_subplot(0, 0, img_matches)

In [ ]:
# Bags of visual words

# cluster all keypoints with kmeans
all_des = np.concatenate((des1, des2, des3, desb), axis=0)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 8
ret, label, center = cv2.kmeans(all_des, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# show points with labels on the image in colors
img1_clusters = img1.copy()
for i in range(len(kp1)):
    # colormap jet
    color = (255 * np.array(plt.cm.jet(label[i,0] / K)[:3])).astype(np.uint8).tolist()
    # overlay points
    cv2.circle(img1_clusters, (int(kp1[i].pt[0]), int(kp1[i].pt[1])), 3, color, -1)

img2_clusters = img2.copy()
i_offset = len(kp1)
for i in range(len(kp2)):
    # colormap jet
    color = (255 * np.array(plt.cm.jet(label[i+i_offset,0] / K)[:3])).astype(np.uint8).tolist()
    # overlay points
    cv2.circle(img2_clusters, (int(kp2[i].pt[0]), int(kp2[i].pt[1])), 3, color, -1)

img3_clusters = img3.copy()
i_offset = len(kp1) + len(kp2)
for i in range(len(kp3)):
    # colormap jet
    color = (255 * np.array(plt.cm.jet(label[i+i_offset,0] / K)[:3])).astype(np.uint8).tolist()
    # overlay points
    cv2.circle(img3_clusters, (int(kp3[i].pt[0]), int(kp3[i].pt[1])), 3, color, -1)

img_bridge_clusters = img_bridge.copy()
i_offset = len(kp1) + len(kp2) + len(kp3)
for i in range(len(kpb)):
    # colormap jet
    color = (255 * np.array(plt.cm.jet(label[i+i_offset,0] / K)[:3])).astype(np.uint8).tolist()
    # overlay points
    cv2.circle(img_bridge_clusters, (int(kpb[i].pt[0]), int(kpb[i].pt[1])), 3, color, -1)

vis = Visualizer(1, 4, figsize=(25,5), title='Clustered SIFT keypoints')
vis.add_image_subplot(0, 0, img1_clusters)
vis.add_image_subplot(0, 1, img2_clusters)
vis.add_image_subplot(0, 2, img3_clusters)
vis.add_image_subplot(0, 3, img_bridge_clusters)


In [ ]:
# Show bag of visual words histograms
h1 = np.histogram(label[:len(kp1)], bins=K)
h2 = np.histogram(label[len(kp1):len(kp1)+len(kp2)], bins=K)
h3 = np.histogram(label[len(kp1)+len(kp2):len(kp1)+len(kp2)+len(kp3)], bins=K)
hb = np.histogram(label[len(kp1)+len(kp2)+len(kp3):], bins=K)

vis = Visualizer(2, 4, figsize=(30,10))
vis.add_image_subplot(0, 0, img1)
vis.add_image_subplot(0, 1, img2)
vis.add_image_subplot(0, 2, img3)
vis.add_image_subplot(0, 3, img_bridge)
vis.add_bar_subplot(1, 0, np.arange(K), h1[0] / np.sum(h1[0]))
vis.add_bar_subplot(1, 1, np.arange(K), h2[0] / np.sum(h2[0]))
vis.add_bar_subplot(1, 2, np.arange(K), h3[0] / np.sum(h3[0]))
vis.add_bar_subplot(1, 3, np.arange(K), hb[0] / np.sum(hb[0]))

# Homography warping example

In [ ]:
img1 = cv2.imread('image_cc_1.jpg')
img3 = cv2.imread('image_cc_3.jpg')

points1 = np.float32([[1801, 1664], [3451, 1692], [3472, 2833], [1726, 2824]]).reshape(4, 2)
points3 = np.float32([[1840, 1955], [2286, 1805], [2263, 2586], [1805, 2592]]).reshape(4, 2)

homography, status = cv2.findHomography(points1, points3)
print(homography)

warped = cv2.warpPerspective(img1, homography, (img3.shape[1], img3.shape[0]))
Visualizer().add_image_subplot(0, 0, warped)


# Laplacian of Gaussian

In [ ]:
# 1D LoG example
simple_signal = (np.abs(np.arange(0, 1000, 1)-500) < 25).astype(np.float32)
sigmas = [0.0125, 0.025, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

vis = Visualizer(8, 3, figsize=(20,25))

for i, sigma in enumerate(sigmas):
    x = 2*np.arange(0, 1000, 1) / 1000.0 - 1
    laplacian_1d = -1/(np.pi*sigma**2) * (1 - x**2 / sigma**2) * np.exp(-x**2 / (2*sigma**2))
    convolved = np.convolve(simple_signal, laplacian_1d, mode='same') * (sigma)

    vis.add_subplot(i, 0, simple_signal, title_str='Input signal')
    vis.add_subplot(i, 1, laplacian_1d, title_str=f'Log Laplacian (sigma={sigma})')
    vis.add_subplot(i, 2, convolved, title_str='Convolved')